# Importing Libraries


In [ ]:
#IT19178400_Ranasinghe R.A.P.T. #IT19119304_Perera J.K.H.A.K. #IT19181820_Akash Y.A.K. #IT19130576_Nanayakkara K.O.K.L.

#Startup Success Prediction Using Random Forest Algorithm

#Perform mathematical and statistical operations 
import numpy as np

#Read data files and create data frames
import pandas as pd

#Import related to visualization
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Split data in to train set and test set
from sklearn.model_selection import train_test_split

In [ ]:
#Import random forest algorithm
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Import classification report 
from sklearn.metrics import classification_report

In [ ]:
#Import of accuracy score
import sklearn.metrics as sm

In [ ]:
#Import recall score and percision score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Importing & Loading the dataset

In [ ]:
#Get data from the churn.csv file to the bank_data dataframe
startup_dataset = pd.read_csv('startup.csv')

In [ ]:
startup_dataset

In [ ]:
#Get an idea about the dataset
startup_dataset.head()

# Dataset Info:

In [ ]:
#Check information related to the dataframe
startup_dataset.info()

In [ ]:
#Check whether if there are any null values
startup_dataset.isnull()

In [ ]:
sum(startup_dataset.isnull().sum())

In [ ]:
#Display Missing values in a grid view
nullValue=pd.DataFrame(startup_dataset.isnull().sum(),columns = ["Null Values"])
nullValue["Missing Values (%)"]=(startup_dataset.isna().sum() / len(startup_dataset)*100)
nullValue = nullValue[nullValue["Missing Values (%)"]  >  0]
nullValue.style.background_gradient(cmap='magma',low =0.3,high=0.2) 

In [ ]:
#Handling Missing Values in 'Unnamed: 6', 'closed_at', 'age_first_milestone_year', 'age_last_milestone_year' columns
startup_dataset['Unnamed: 6'] = startup_dataset.apply(lambda row: (row.city) + " " + (row.state_code) + " " +(row.zip_code)  , axis = 1)

startup_dataset['closed_at'] = startup_dataset['closed_at'].fillna(value="31/12/2013")

startup_dataset['age_first_milestone_year'] = startup_dataset['age_first_milestone_year'].fillna(value="0")

startup_dataset['age_last_milestone_year'] = startup_dataset['age_last_milestone_year'].fillna(value="0")

In [ ]:
#Handling Missing Values in 'state_code.1' column
for index, row in startup_dataset.iterrows():
    if row['state_code']!=row['state_code.1']:
        print(index, row['state_code'], row['state_code.1'])
        
startup_dataset.drop(["state_code.1"], axis=1, inplace=True)

In [ ]:
#Graphical view of null values in the whole dataframe
sns.heatmap(startup_dataset.isnull(),yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
#Numerical value of the total number of null values in the data set
total_null_values = sum(startup_dataset.isnull().sum())
print(total_null_values)

In [ ]:
#Futher checking for null values 
print(startup_dataset.isnull().sum())

In [ ]:
#Changing 'status' data value
startup_dataset['status'] = startup_dataset.status.map({'acquired':1, 'closed':0})

#Handling 'status' data type to int
startup_dataset['status'].astype(int)

In [ ]:
#Handling 'age_first_milestone_year' and 'age_last_milestone_year' data type to float
startup_dataset['age_first_milestone_year'] = startup_dataset.age_first_milestone_year.astype(float)
startup_dataset['age_last_milestone_year'] = startup_dataset.age_last_milestone_year.astype(float)

In [ ]:
#Visualize the count of startup status
sns.countplot(x='status', data=startup_dataset)

In [ ]:
#graphical representation related to category which has the largest no of startup
fig, axi = plt.subplots(figsize=(12,8))

_ = sns.countplot(x="category_code", hue="status", data=startup_dataset,
              order=startup_dataset.category_code.value_counts().index)

_ = axi.set_xticklabels(axi.get_xticklabels(), rotation=90)
_ = axi.set(xlabel="Category", ylabel="Number of startup")
plt.legend(bbox_to_anchor=(0.946, 0.91))


In [ ]:
#Visualize the category which has the largest number Success Rate

startupData1 = startup_dataset[startup_dataset['status']==1].groupby(['category_code']).agg({'status':'count'}).reset_index()
startupData1.columns=['category_code','total_success']

startupData2 = startup_dataset[startup_dataset['status']==0].groupby(['category_code']).agg({'status':'count'}).reset_index()
startupData2.columns=['category_code','total_closed']

startupData3=startup_dataset.groupby(['category_code']).agg({'status':'count'}).reset_index()
startupData3.columns=['category_code','total_startup']

startupData1= startupData1.merge(startupData2, on='category_code')
startupData1= startupData1.merge(startupData3, on='category_code')

startupData1['success_rate']= round((startupData1['total_success'] / startupData1['total_startup']) * 100,2)

most_succes_rate = startupData1.sort_values('success_rate', ascending=False)
most_succes_rate

fig, ax = plt.subplots(figsize=(10,7))
_ = sns.barplot(x="category_code", y="success_rate", data=most_succes_rate,
                palette="nipy_spectral", ax=ax)
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(xlabel="Category", ylabel="Success Rate of Start Up")

In [ ]:
#graphical representation related to category has the largest number Success Rate
sns.countplot(x='status',hue='is_top500', data=startup_dataset)

In [ ]:
#New column named as "has_RoundABCD"
startup_dataset['has_RoundABCD'] = np.where((startup_dataset['has_roundA'] == 1) | (startup_dataset['has_roundB'] == 1) | (startup_dataset['has_roundC'] == 1) | (startup_dataset['has_roundD'] == 1), 1, 0)

#New column named as "has_Investor"
startup_dataset['has_Investor'] = np.where((startup_dataset['has_VC'] == 1) | (startup_dataset['has_angel'] == 1), 1, 0)

#New column named as "invalid_startup"
startup_dataset['invalid_startup'] = np.where((startup_dataset['has_RoundABCD'] == 0) & (startup_dataset['has_VC'] == 0) & (startup_dataset['has_angel'] == 0), 1, 0)

#New column named as "has_Seed"
startup_dataset['has_Seed'] = np.where((startup_dataset['has_RoundABCD'] == 0) & (startup_dataset['has_Investor'] == 1), 1, 0)

startup_dataset.head()


# Data Preprocessing

In [ ]:
#Drop unwanted variables
startup_dataset.drop(['state_code','id','Unnamed: 6','category_code','object_id','zip_code','founded_at','closed_at','first_funding_at','last_funding_at','city','name','Unnamed: 0','latitude','longitude','relationships'], axis=1, inplace=True)


In [ ]:
#View data set
startup_dataset

In [ ]:
#Get the copy of original data set as a training data set
lbEncd = LabelEncoder()

In [ ]:
startup_data_tf = startup_dataset.copy()

for i in startup_dataset.columns :
    startup_data_tf[i] = lbEncd.fit_transform(startup_dataset[i])

In [ ]:
startup_data_tf

# Features and Labels

In [ ]:
#Dataframe separate in to features and labels
X = startup_data_tf.drop('status', axis=1)
Y = startup_data_tf['status']

# Train Test Split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
#Print slpited data set
print(X_train.shape, '-> Training Feature Shape')
print(X_test.shape, '-> Testing Feature Shape')
print(Y_train.shape, '-> Training Labels Shape')
print(Y_test.shape, '-> Testing Labels Shape')

# Build a Model

In [ ]:
#Create the model
startup_model = RandomForestClassifier(bootstrap=False, max_depth=12, min_samples_leaf=100, min_samples_split=20,
                       n_estimators=70,criterion='entropy',n_jobs=1000,max_leaf_nodes=100,max_features='auto')